In [1]:
# Import needed libraries 

import pandas as pd 
import os
import datetime
from datetime import timedelta
import numpy as np 
from scipy.signal import argrelextrema
import alpaca_trade_api as tradeapi 
import matplotlib.pyplot as plt 
import matplotlib.dates as mpdates
from mplfinance.original_flavor import candlestick_ohlc
from dotenv import load_dotenv
from itertools import islice
from pandas .tseries.offsets import DateOffset

#Import various libraries needed to train and test ML models
from keras.datasets import imdb
from keras.layers import LSTM, Dense, Embedding
from keras.preprocessing import sequence
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [2]:
# Load .env file

load_dotenv()

True

In [3]:
# Set Alpaca API key and secret
ALPACA_API_KEY = os.getenv("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")
ALPACA_ENDPOINT_KEY = os.getenv("ALPACA_END_POINT")


# Create the Alpaca API object
api = tradeapi.REST(
    ALPACA_API_KEY,
    ALPACA_SECRET_KEY,
    api_version="v2",
    base_url= ALPACA_ENDPOINT_KEY
)

In [4]:
# Parameters for Stock Data from Alpacas
# Establish time frame (5 minute)

time_frame = "5min"

# Identify what stock symbol is trading

stock_symbol = "QQQ"

# Identify what start date to begin data analysis

start_date = pd.Timestamp("2021-02-01", tz="America/New_York").isoformat()

# Identify what end date to finalize data analysis

end_date = pd.Timestamp("2021-06-01", tz="America/New_York").isoformat()

In [5]:
# Function to call daily stock data

def get_stock_data(api, stock_symbol, time_frame, start_date, end_date):
    
    # Assuming api.get_bars returns a DataFrame with a 'df' attribute
    
    stock_data = api.get_bars(
        stock_symbol, 
        time_frame, 
        start=start_date, 
        end=end_date
        ).df
    return stock_data

stock_data = get_stock_data(api, stock_symbol, time_frame, start_date, end_date)

# Displays the information pulled for working through code

stock_data.info()
display(stock_data.head())
display(stock_data.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15458 entries, 2021-02-01 09:00:00+00:00 to 2021-05-28 21:55:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   close        15458 non-null  float64
 1   high         15458 non-null  float64
 2   low          15458 non-null  float64
 3   trade_count  15458 non-null  int64  
 4   open         15458 non-null  float64
 5   volume       15458 non-null  int64  
 6   vwap         15458 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 966.1 KB


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2021-02-01 09:00:00+00:00,317.78,317.79,317.26,457,317.62,152785,317.615546
2021-02-01 09:05:00+00:00,317.63,317.79,317.54,42,317.75,5968,317.621602
2021-02-01 09:10:00+00:00,317.85,317.90,317.65,41,317.65,5138,317.836724
2021-02-01 09:15:00+00:00,318.15,318.25,317.97,77,318.00,18913,318.110307
2021-02-01 09:20:00+00:00,318.01,318.33,318.00,88,318.11,20816,318.259888


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2021-05-28 21:25:00+00:00,333.84,333.84,333.80,12,333.80,788,333.814201
2021-05-28 21:30:00+00:00,333.80,333.82,333.80,15,333.82,2008,333.814337
2021-05-28 21:45:00+00:00,333.84,333.84,333.82,11,333.82,1115,333.827005
2021-05-28 21:50:00+00:00,333.83,333.83,333.82,6,333.82,702,333.828604
2021-05-28 21:55:00+00:00,333.82,333.83,333.82,11,333.83,1023,333.823030


In [6]:
# Function to prepare daily stock data to identify double top/bottom patterns and prepare for targets
# Includes establishing a polynomial fit and assigning new columns for localized min/max

# Polynomial Degree

polynomial_degree = 25

def polynomial_min_max_fit(stock_data, polynomial_degree):
    
    min_length = len(stock_data.index)
    x_data = np.arange(min_length)

    # Polynomial fitting
    polynomial_coefficients_open = np.polyfit(x_data, stock_data['open'][:min_length], polynomial_degree)
    polynomial_coefficients_high = np.polyfit(x_data, stock_data['high'][:min_length], polynomial_degree)
    polynomial_coefficients_low = np.polyfit(x_data, stock_data['low'][:min_length], polynomial_degree)
    polynomial_coefficients_close = np.polyfit(x_data, stock_data['close'][:min_length], polynomial_degree)

    # Evaluate the polynomial fit for plotting
    y_polynomial_open = np.polyval(polynomial_coefficients_open, x_data)
    y_polynomial_high = np.polyval(polynomial_coefficients_high, x_data)
    y_polynomial_low = np.polyval(polynomial_coefficients_low, x_data)
    y_polynomial_close = np.polyval(polynomial_coefficients_close, x_data)

    # Identify local extrema for polynomial fit data (minima and maxima)
    local_poly_minima = argrelextrema(y_polynomial_close, np.less, order=5)[0]
    local_poly_maxima = argrelextrema(y_polynomial_close, np.greater, order=5)[0]

    # Convert the close price polynomial fit data into a dataframe
    # This is done for OHLC poly fit data

    poly_df = pd.DataFrame(y_polynomial_open)
    columns = ['poly_fit_open']
    poly_df.columns = columns
    poly_df = poly_df.assign(poly_fit_high = y_polynomial_high)
    poly_df = poly_df.assign(poly_fit_low = y_polynomial_low)
    poly_df = poly_df.assign(poly_fit_close = y_polynomial_close)

    # Reset the index of the original updated ticker dataframe to concat with the polynomial dataframe that does not include a timeseries
    # This will ensure that the indexed intergers of the ploynomial fit align with the time each data point corresponds to

    updated_stock_data = stock_data.reset_index()
    updated_stock_data = pd.concat([updated_stock_data, poly_df], axis='columns', join='inner')
    updated_stock_data.head()

    # Add minima and maxima column to the DataFrame

    updated_stock_data["minima"] = 0
    updated_stock_data["maxima"] = 0
    updated_stock_data.head()

    # Mark rows with local minima as 1 in the 'minima' column

    for index in local_poly_minima:
        updated_stock_data.at[index, 'minima'] = -1

    for index in local_poly_maxima:
        updated_stock_data.at[index, "maxima"] = 1

    # Create Target Columns - Double Top Target & Double Bottom Target
        
    updated_stock_data["dbl_top_target"] = 0
    updated_stock_data["dbl_bot_target"] = 0

    # # Plot data: COMMENTED OUT TO NOT REPEAT PLOTS AT END OF PROGRAM
    # # Plot the stock data and identified minima

    # plt.figure(figsize=(15, 5))
    # plt.plot(stock_data.index, stock_data["close"], label='Close Prices', alpha=0.7)

    # # Plot polynomial fit

    # plt.plot(stock_data.index[:min_length], y_polynomial_close, '-', markersize=1.0, color='black', alpha=0.9, label='Polynomial Fit')

    # # Plot red dots at local minima and blue dots at local maxima

    # plt.scatter(stock_data.index[local_poly_minima], y_polynomial_close[local_poly_minima], color='red', label='Local Minima')
    # plt.scatter(stock_data.index[local_poly_maxima],y_polynomial_close[local_poly_maxima], color="blue", label = "Local Maxima")
    
    return updated_stock_data

# For data checking/confirmation
updated_stock_data = polynomial_min_max_fit(stock_data,polynomial_degree)
updated_stock_data.head()

C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_32980\2367383943.py:14: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_open = np.polyfit(x_data, stock_data['open'][:min_length], polynomial_degree)
C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_32980\2367383943.py:15: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_high = np.polyfit(x_data, stock_data['high'][:min_length], polynomial_degree)
C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_32980\2367383943.py:16: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_low = np.polyfit(x_data, stock_data['low'][:min_length], polynomial_degree)
C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_32980\2367383943.py:17: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_close = np.polyfit(x_data, stock_data['close'][:min_length], polynomial_degree)


,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,minima,maxima,dbl_top_target,dbl_bot_target
0,2021-02-01 09:00:00+00:00,317.78,317.79,317.26,457,317.62,152785,317.615546,312.930406,313.157730,312.744286,312.916258,0,0,0,0
1,2021-02-01 09:05:00+00:00,317.63,317.79,317.54,42,317.75,5968,317.621602,313.056023,313.282869,312.869895,313.042180,0,0,0,0
2,2021-02-01 09:10:00+00:00,317.85,317.90,317.65,41,317.65,5138,317.836724,313.180801,313.407171,312.994663,313.167260,0,0,0,0
3,2021-02-01 09:15:00+00:00,318.15,318.25,317.97,77,318.00,18913,318.110307,313.304743,313.530640,313.118597,313.291503,0,0,0,0
4,2021-02-01 09:20:00+00:00,318.01,318.33,318.00,88,318.11,20816,318.259888,313.427853,313.653279,313.241700,313.414912,0,0,0,0


In [7]:
# Define the time independent DF for double top/bottom identification
# Timestamp remains a column, but can identify pattern from peak to peak immediately

def time_independent_data(updated_stock_data):

    # Define and establish time independent DF

    time_independent_df = []
    time_independent_df = pd.DataFrame(time_independent_df, columns = ["timestamp","close","high","low","trade_count","open","volume","vwap","poly_fit_open","poly_fit_high","poly_fit_low","poly_fit_close","minima","maxima", "dbl_top_target","dbl_bot_target"])

    # Loop to iterate through all rows of stock data and write min/max to new DF

    for index, row in islice(updated_stock_data.iterrows(), 0, None):

        # Assign to rows only those that contain local min or max

        if (updated_stock_data.at[index,"minima"] == -1):
            time_independent_df.loc[index] = row
            a = updated_stock_data.iloc[index]["poly_fit_close"]
        elif (updated_stock_data.at[index,"maxima"] == 1):
            time_independent_df.loc[index] = row
            a = updated_stock_data.iloc[index]["poly_fit_close"]

    # Reset time dependent index
    # Set new time independent index

    time_independent_df.reset_index(inplace = True)
    time_independent_df.rename(columns={"index":"time_dependent_index"}, inplace = True)

    return time_independent_df

# Data confirmation/check

time_independent_df = time_independent_data(updated_stock_data)
time_independent_df.head(10)

,time_dependent_index,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,minima,maxima,dbl_top_target,dbl_bot_target
0,1536,2021-02-11 11:50:00+00:00,334.1,334.22,334.06,51,334.16,5938,334.109267,335.489916,335.593558,335.376124,335.490674,0,1,0,0
1,4424,2021-03-05 19:40:00+00:00,307.535,308.24,307.14,9474,307.99,897074,307.727024,309.331801,309.647273,309.01703,309.327414,-1,0,0,0
2,5954,2021-03-17 22:20:00+00:00,322.15,322.22,322.15,40,322.22,8299,322.186672,318.284073,318.479015,318.075503,318.280644,0,1,0,0
3,7088,2021-03-25 23:00:00+00:00,311.59,311.64,311.54,18,311.58,3860,311.613834,313.906709,314.095617,313.700954,313.903994,-1,0,0,0
4,9364,2021-04-14 10:05:00+00:00,341.19,341.26,341.18,30,341.23,2466,341.214217,339.617838,339.733633,339.493057,339.617857,0,1,0,0
5,10262,2021-04-20 23:20:00+00:00,335.1,335.13,335.09,16,335.11,2150,335.111153,337.793903,337.933293,337.649807,337.791455,-1,0,0,0
6,11234,2021-04-28 11:05:00+00:00,340.03,340.06,339.86,93,339.96,20839,339.952775,339.996713,340.109649,339.872701,339.999674,0,1,0,0
7,13705,2021-05-17 16:35:00+00:00,323.24,323.27,322.7099,1683,322.73,169100,323.045953,321.629896,321.825519,321.441034,321.629637,-1,0,0,0
8,14951,2021-05-26 11:05:00+00:00,334.12,334.24,334.04,62,334.24,9224,334.163679,333.620895,333.724459,333.513186,333.61944,0,1,0,0
9,15103,2021-05-26 23:45:00+00:00,333.98,333.99,333.93,41,333.95,10923,333.962246,333.520122,333.622286,333.41438,333.521461,-1,0,0,0


In [8]:
# Function to identify daily double top/bottom patterns
# Inherently there should only be 1 identification of the pattern as it is considered a trend reversal pattern
# The loop will break once the pattern is identified

def identify_double_patterns(time_independent_df, updated_stock_data):

    # Initialize variables to identify double top/bottom patterns
    # Time dependent variable x_0 will always begin at a local min/max which also coincides
    # With the start of a trend into a potential double top/bottom
    
    x_0 = 0

    # Initiation of double top/bottom variable
    
    a = 0

    # First peak/valley of double top/bottom pattern
    
    b = 0

    # Trough/peak of double top/bottom pattern

    c = 0

    # Second peak/vallye of double top/bottom pattern
    
    d = 0

    # Trigger of double top/bottom pattern
    
    e = 0

    # Final time increment to finalize and trigger double top signal
    
    x_f = 0

    # Read through code to identify double top/bottom and assign to target columns.

    for index, row in islice(time_independent_df.iterrows(), 0, len(time_independent_df) - 4):

        # Check for double top
        # If found then assigns x_f final time for writing to targets in time dependent dataframe

        if (time_independent_df.at[index,"minima"] == -1):
            a = time_independent_df.iloc[index]["poly_fit_close"]
            b = time_independent_df.iloc[index + 1]["poly_fit_close"]
            c = time_independent_df.iloc[index + 2]["poly_fit_close"]
            d = time_independent_df.iloc[index + 3]["poly_fit_close"]
            e = time_independent_df.iloc[index + 4]["poly_fit_close"]
            x_0 = time_independent_df.iloc[index]["time_dependent_index"]

            if (time_independent_df.iloc[index + 1]["poly_fit_low"])*.95 < d < (time_independent_df.iloc[index + 1]["poly_fit_high"])*1.05 and (e <= c):
                x_f = time_independent_df.iloc[index + 4]["time_dependent_index"]
                updated_stock_data.at[x_f, "long_short_target"] = -1
                while (x_0 < x_f + 1):
                    updated_stock_data.at[x_0, "dbl_top_target"] = 1
                    x_0 = x_0 + 1
                break
            
            

        # Check for double bottom
        # If found then assigns x_f final time for writing to targets in time dependent dataframe
            
        elif (time_independent_df.at[index,"maxima"] == 1):
            a = time_independent_df.iloc[index]["poly_fit_close"]
            b = time_independent_df.iloc[index + 1]["poly_fit_close"]
            c = time_independent_df.iloc[index + 2]["poly_fit_close"]
            d = time_independent_df.iloc[index + 3]["poly_fit_close"]
            e = time_independent_df.iloc[index + 4]["poly_fit_close"]
            x_0 = time_independent_df.iloc[index]["time_dependent_index"] 

            if (time_independent_df.iloc[index+1]["poly_fit_low"])*.95 < d < (time_independent_df.iloc[index+1]["poly_fit_high"])*1.05 and (e >= c):
                x_f_min = time_independent_df.iloc[index + 4]["time_dependent_index"]
                updated_stock_data.at[x_f, "long_short_target"] = 1
                while (x_0 < x_f + 1):
                    updated_stock_data.at[x_0, "dbl_bot_target"] = 1
                    x_0 = x_0 + 1
                break
            
    return updated_stock_data, x_f

updated_stock_data, x_f = identify_double_patterns(time_independent_df,updated_stock_data)

#print(x_f)

In [9]:
# Loop to collect and assess daily data one day at a time

master_df = {
    'timestamp': [],  # List of timestamps
    'close': [],      # List of close prices
    'high': [],       # List of high prices
    'low': [],        # List of low prices
    'trade_count': [], # List of trade counts
    'open': [],       # List of open prices
    'volume': [],     # List of volumes
    'vwap': [],       # List of volume-weighted average prices
    'poly_fit_open': [],    # List of polynomial fit open prices
    'poly_fit_high': [],    # List of polynomial fit high prices
    'poly_fit_low': [],     # List of polynomial fit low prices
    'poly_fit_close': [],   # List of polynomial fit close prices
    'minima': [],           # List of minima
    'maxima': [],           # List of maxima
    'dbl_top_target': [],   # List of double top targets
    'dbl_bot_target': [],    # List of double bottom targets
    'long_short_target': []       # List of ML Model target
}

daily_dataframes = []

master_df = pd.DataFrame(master_df)

current_date = pd.to_datetime(start_date)
next_day_date = current_date + pd.offsets.BDay(1)

print(current_date)
print(next_day_date)

while current_date <= (pd.to_datetime(end_date)):

    # next_day_date = next_day_date + pd.offsets.BDay(1)
    # current_date = current_date + pd.offsets.BDay(1)
    # Convert current_date and next_day_date to ISO format for API call

    current_date_iso = current_date.isoformat()
    # next_day_date = current_date + pd.offsets.BDay(1)
    next_day_date_iso = next_day_date.isoformat()

    #current_date = datetime.datetime.fromisoformat(current_date_iso) + pd.offsets.BDay(1)
    
    # Fetch stock data for the current day

    stock_data = get_stock_data(api, stock_symbol, time_frame, current_date_iso, next_day_date_iso)

    if len(stock_data.index) == 0:
        next_day_date = next_day_date + pd.offsets.BDay(1)
        current_date = current_date + pd.offsets.BDay(1)
        continue
    
    # Run polynomial fit function for updated stock data

    updated_stock_data = polynomial_min_max_fit(stock_data, polynomial_degree)
    
    # Isolate Mins/maxes

    time_independent_df = time_independent_data(updated_stock_data)

    # Updated stock data with double tops/bots identified by function
    # x_f is the double top/bot trigger row(index) for current day's double top/bot

    updated_stock_data, x_f = identify_double_patterns(time_independent_df, updated_stock_data)
    
    # Perform your analysis or call your functions here
    # e.g., identify_double_patterns(time_independent_df, updated_stock_data)

    daily_dataframes.append(updated_stock_data)

    master_df = pd.concat(daily_dataframes, ignore_index=True)
    # display(master_df.tail())
    # Increment to the next day

    next_day_date = next_day_date + pd.offsets.BDay(1)
    current_date = current_date + pd.offsets.BDay(1)
    # next_day_date = pd.Timestamp(next_day_date_iso) + pd.offsets.BDay(1)
    
    # print(current_date)
    # print(next_day_date)

2021-02-01 00:00:00-05:00
2021-02-02 00:00:00-05:00


C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_32980\2367383943.py:14: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_open = np.polyfit(x_data, stock_data['open'][:min_length], polynomial_degree)
C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_32980\2367383943.py:15: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_high = np.polyfit(x_data, stock_data['high'][:min_length], polynomial_degree)
C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_32980\2367383943.py:16: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_low = np.polyfit(x_data, stock_data['low'][:min_length], polynomial_degree)
C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_32980\2367383943.py:17: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_close = np.polyfit(x_data, stock_data['close'][:min_length], polynomial_degree)
C:\Users\JamesDavidson\AppData\Local\Temp\ipykernel_32980\2367383943.py:14: RankWarning: Pol

In [10]:
# Shift master dataframe to lag by one datapoint for training ML model 

master_df['close_lagged'] = master_df['close'].shift(1)
master_df['high_lagged'] = master_df['high'].shift(1)
master_df['low_lagged'] = master_df['low'].shift(1)
master_df['open_lagged'] = master_df['open'].shift(1)
master_df['trade_count_lagged'] = master_df['trade_count'].shift(1)
master_df['volume_lagged'] = master_df['volume'].shift(1)
master_df['vwap_lagged'] = master_df['vwap'].shift(1)
master_df['poly_open_lagged'] = master_df['poly_fit_open'].shift(1)
master_df['poly_high_lagged'] = master_df['poly_fit_high'].shift(1)
master_df['poly_low_lagged'] = master_df['poly_fit_low'].shift(1)
master_df['poly_close_lagged'] = master_df['poly_fit_close'].shift(1)
master_df['minima_lagged'] = master_df['minima'].shift(1)
master_df['maxima_lagged'] = master_df['maxima'].shift(1)
master_df['dbl_top_target_lagged'] = master_df['dbl_top_target'].shift(1)
master_df['dbl_bot_target_lagged'] = master_df['dbl_bot_target'].shift(1)
master_df['long_short_target_lagged'] = master_df['long_short_target'].shift(1)

In [11]:
# Convert the NaN values within both tagret columns into 0's to feed into ML Models
master_df['long_short_target'] = master_df['long_short_target'].fillna(0)
master_df['long_short_target_lagged'] = master_df['long_short_target_lagged'].fillna(0)
master_df.dropna()

master_df.info()
print(len(master_df))
print(type(master_df))
print(sum(master_df["dbl_top_target"]))
print(sum(master_df["dbl_bot_target"]))
print(sum(master_df["long_short_target"]))
display(master_df.head())
display(master_df.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15458 entries, 0 to 15457
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   timestamp                 15458 non-null  datetime64[ns, UTC]
 1   close                     15458 non-null  float64            
 2   high                      15458 non-null  float64            
 3   low                       15458 non-null  float64            
 4   trade_count               15458 non-null  int64              
 5   open                      15458 non-null  float64            
 6   volume                    15458 non-null  int64              
 7   vwap                      15458 non-null  float64            
 8   poly_fit_open             15458 non-null  float64            
 9   poly_fit_high             15458 non-null  float64            
 10  poly_fit_low              15458 non-null  float64            
 11  poly_fit_close 

,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,...,vwap_lagged,poly_open_lagged,poly_high_lagged,poly_low_lagged,poly_close_lagged,minima_lagged,maxima_lagged,dbl_top_target_lagged,dbl_bot_target_lagged,long_short_target_lagged
0,2021-02-01 09:00:00+00:00,317.78,317.79,317.26,457,317.62,152785,317.615546,317.589178,317.713873,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2021-02-01 09:05:00+00:00,317.63,317.79,317.54,42,317.75,5968,317.621602,317.741114,317.920427,...,317.615546,317.589178,317.713873,317.194772,317.668435,0.0,0.0,0.0,0.0,0.0
2,2021-02-01 09:10:00+00:00,317.85,317.90,317.65,41,317.65,5138,317.836724,317.855104,318.045420,...,317.621602,317.741114,317.920427,317.621386,317.859682,0.0,0.0,0.0,0.0,0.0
3,2021-02-01 09:15:00+00:00,318.15,318.25,317.97,77,318.00,18913,318.110307,317.933155,318.110569,...,317.836724,317.855104,318.045420,317.816195,317.940566,0.0,0.0,0.0,0.0,0.0
4,2021-02-01 09:20:00+00:00,318.01,318.33,318.00,88,318.11,20816,318.259888,317.980292,318.134466,...,318.110307,317.933155,318.110569,317.885216,317.961841,0.0,1.0,0.0,0.0,0.0


,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,...,vwap_lagged,poly_open_lagged,poly_high_lagged,poly_low_lagged,poly_close_lagged,minima_lagged,maxima_lagged,dbl_top_target_lagged,dbl_bot_target_lagged,long_short_target_lagged
15453,2021-05-28 21:25:00+00:00,333.84,333.84,333.80,12,333.80,788,333.814201,333.788918,333.799171,...,333.920281,333.781296,333.802876,333.797115,333.806531,0.0,0.0,0.0,0.0,0.0
15454,2021-05-28 21:30:00+00:00,333.80,333.82,333.80,15,333.82,2008,333.814337,333.806065,333.813151,...,333.814201,333.788918,333.799171,333.802676,333.818287,0.0,0.0,0.0,0.0,0.0
15455,2021-05-28 21:45:00+00:00,333.84,333.84,333.82,11,333.82,1115,333.827005,333.829971,333.846285,...,333.814337,333.806065,333.813151,333.805815,333.830350,0.0,0.0,0.0,0.0,0.0
15456,2021-05-28 21:50:00+00:00,333.83,333.83,333.82,6,333.82,702,333.828604,333.846077,333.873472,...,333.827005,333.829971,333.846285,333.809184,333.838308,0.0,1.0,0.0,0.0,0.0
15457,2021-05-28 21:55:00+00:00,333.82,333.83,333.82,11,333.83,1023,333.823030,333.816180,333.808664,...,333.828604,333.846077,333.873472,333.815228,333.835725,0.0,0.0,0.0,0.0,0.0


In [12]:
master_df = master_df.drop(0)
master_df.head()

,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,...,vwap_lagged,poly_open_lagged,poly_high_lagged,poly_low_lagged,poly_close_lagged,minima_lagged,maxima_lagged,dbl_top_target_lagged,dbl_bot_target_lagged,long_short_target_lagged
1,2021-02-01 09:05:00+00:00,317.63,317.79,317.54,42,317.75,5968,317.621602,317.741114,317.920427,...,317.615546,317.589178,317.713873,317.194772,317.668435,0.0,0.0,0.0,0.0,0.0
2,2021-02-01 09:10:00+00:00,317.85,317.90,317.65,41,317.65,5138,317.836724,317.855104,318.045420,...,317.621602,317.741114,317.920427,317.621386,317.859682,0.0,0.0,0.0,0.0,0.0
3,2021-02-01 09:15:00+00:00,318.15,318.25,317.97,77,318.00,18913,318.110307,317.933155,318.110569,...,317.836724,317.855104,318.045420,317.816195,317.940566,0.0,0.0,0.0,0.0,0.0
4,2021-02-01 09:20:00+00:00,318.01,318.33,318.00,88,318.11,20816,318.259888,317.980292,318.134466,...,318.110307,317.933155,318.110569,317.885216,317.961841,0.0,1.0,0.0,0.0,0.0
5,2021-02-01 09:25:00+00:00,318.11,318.16,318.00,42,318.08,7704,318.125444,318.002786,318.132133,...,318.259888,317.980292,318.134466,317.893533,317.954719,0.0,0.0,0.0,0.0,0.0


In [13]:
#groupby day and keep 5 minute increments
grouped_df = master_df.groupby(master_df["timestamp"].dt.date).apply(lambda x:x).reset_index(drop=True)
grouped_df.set_index("timestamp",inplace=True)
grouped_df.head()

,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,...,vwap_lagged,poly_open_lagged,poly_high_lagged,poly_low_lagged,poly_close_lagged,minima_lagged,maxima_lagged,dbl_top_target_lagged,dbl_bot_target_lagged,long_short_target_lagged
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-02-01 09:05:00+00:00,317.63,317.79,317.54,42,317.75,5968,317.621602,317.741114,317.920427,317.621386,...,317.615546,317.589178,317.713873,317.194772,317.668435,0.0,0.0,0.0,0.0,0.0
2021-02-01 09:10:00+00:00,317.85,317.90,317.65,41,317.65,5138,317.836724,317.855104,318.045420,317.816195,...,317.621602,317.741114,317.920427,317.621386,317.859682,0.0,0.0,0.0,0.0,0.0
2021-02-01 09:15:00+00:00,318.15,318.25,317.97,77,318.00,18913,318.110307,317.933155,318.110569,317.885216,...,317.836724,317.855104,318.045420,317.816195,317.940566,0.0,0.0,0.0,0.0,0.0
2021-02-01 09:20:00+00:00,318.01,318.33,318.00,88,318.11,20816,318.259888,317.980292,318.134466,317.893533,...,318.110307,317.933155,318.110569,317.885216,317.961841,0.0,1.0,0.0,0.0,0.0
2021-02-01 09:25:00+00:00,318.11,318.16,318.00,42,318.08,7704,318.125444,318.002786,318.132133,317.878399,...,318.259888,317.980292,318.134466,317.893533,317.954719,0.0,0.0,0.0,0.0,0.0


In [14]:
# Create feature and testing sets
X_columns_to_drop = ['long_short_target', 'long_short_target_lagged']
X = grouped_df.drop(columns=X_columns_to_drop)

y = grouped_df['long_short_target_lagged']

In [15]:
# Prepare sets into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [16]:
# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed

In [17]:
# Instantiate the KNeighborsClassifier model with n_neighbors = 3 
knn = KNeighborsClassifier(n_neighbors=3)

In [18]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


KNeighborsClassifier(n_neighbors=3)

In [19]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [20]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.93      0.84      0.88        31
         0.0       1.00      1.00      1.00     10767
         1.0       0.00      0.00      0.00        22

    accuracy                           1.00     10820
   macro avg       0.64      0.61      0.63     10820
weighted avg       1.00      1.00      1.00     10820



c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd

In [21]:
# Declare a logistic regression model.
# Apply a random_state of 9 to the model
logistic_regression_model = LogisticRegression(random_state=9)

# Fit and save the logistic regression model using the training data
lr_model = logistic_regression_model.fit(X_train, y_train)

c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed

In [22]:
#Generate training predictions
training_predictions = lr_model.predict(X_train)

#Generate testing predictions
testing_predictions = logistic_regression_model.predict(X_test)

c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed

In [23]:
# Import the model for sklearn's confusion matrix
from sklearn.metrics import confusion_matrix

# Create and save the confustion matrix for the training data
training_matrix = confusion_matrix(y_train, training_predictions)

# Print the confusion matrix for the training data
print(training_matrix)

[[   0   11    0]
 [   0 4610    0]
 [   0   16    0]]


c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd

In [24]:
# Create and save the confustion matrix for the testing data
test_matrix = confusion_matrix(y_test, testing_predictions)

# Print the confusion matrix for the testing data
print(test_matrix)

[[    0    31     0]
 [    1 10766     0]
 [    0    22     0]]


c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd

In [25]:
# Create and save the training classifiction report
training_report = classification_report(y_train, training_predictions)

# Print the training classification report
print(training_report)

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00        11
         0.0       0.99      1.00      1.00      4610
         1.0       0.00      0.00      0.00        16

    accuracy                           0.99      4637
   macro avg       0.33      0.33      0.33      4637
weighted avg       0.99      0.99      0.99      4637



c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd

In [32]:
from sklearn.svm import SVC

parameters = {
    'kernel': ('linear', 'rbf'),
    'C': [1, 5, 15, 20],
    'degree': [1, 5, 10, 15]
}

svc = SVC()

In [34]:
from sklearn import datasets

iris = datasets.load_iris()
X_iris = iris.data
y_iris = iris.target

In [35]:
clf = GridSearchCV(svc, parameters)

clf.fit(iris.data, iris.target)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 5, 15, 20], 'degree': [1, 5, 10, 15],
                         'kernel': ('linear', 'rbf')})

In [36]:
clf.cv_results_

{'mean_fit_time': array([0.00041475, 0.00060334, 0.00114312, 0.00050149, 0.00101795,
        0.00125756, 0.00070324, 0.00083666, 0.00103688, 0.00061989,
        0.00107937, 0.00080099, 0.00070329, 0.00040202, 0.00080128,
        0.00040016, 0.00070424, 0.00082412, 0.00085745, 0.00051789,
        0.0007051 , 0.00080261, 0.00059972, 0.00075655, 0.00040078,
        0.00080271, 0.00060129, 0.00092998, 0.00084577, 0.00070138,
        0.00095925, 0.00070839]),
 'std_fit_time': array([5.08510928e-04, 8.01895670e-04, 1.94444967e-04, 4.47771054e-04,
        5.40415386e-04, 3.43647796e-04, 5.10179853e-04, 4.22848841e-04,
        7.51166102e-05, 5.07090951e-04, 5.77901562e-04, 4.00508629e-04,
        4.00405656e-04, 4.92381560e-04, 4.00659209e-04, 4.90100589e-04,
        4.01676409e-04, 4.12857155e-04, 4.38781516e-04, 6.44489201e-04,
        4.01474534e-04, 4.01331506e-04, 4.89668756e-04, 6.53384216e-04,
        3.72656919e-04, 5.10586809e-04, 4.90960023e-04, 4.82261716e-04,
        4.30926834e-0

In [26]:
# Add these models to the list of models to test
models_to_test = [
    {
        'model': KNeighborsClassifier(),
        'param_grid': {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}
    },
    {
        'model': LogisticRegression(),
        'param_grid': {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}
    },
]

In [27]:
# Run and iterte for each model using GridSearch
for model_info in models_to_test:
    grid_search = GridSearchCV(model_info['model'], model_info['param_grid'], cv=5)
    grid_search.fit(X_train, y_train)
    print(f"Best parameters for {model_info['model']}:", grid_search.best_params_)
    print(f"Best score: {grid_search.best_score_}")

c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd

Best parameters for KNeighborsClassifier(): {'n_neighbors': 3, 'weights': 'uniform'}
Best score: 0.9941775471487556


c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future 

Best parameters for LogisticRegression(): {'C': 0.1, 'penalty': 'l2'}
Best score: 0.9941775471487556


c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [28]:
results_list = []  # List to store results from each GridSearchCV
for model_info in models_to_test:
    grid_search = GridSearchCV(model_info['model'], model_info['param_grid'], cv=5)
    grid_search.fit(X_train, y_train)
    results_list.append({
        'model': model_info['model'].__class__.__name__,
        'params': grid_search.best_params_,
        'mean_score': grid_search.best_score_,
        'cv_results': grid_search.cv_results_
    })

c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\JamesDavidson\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd

In [29]:
# Combine results into a DataFrame
results_df = pd.DataFrame(results_list)


In [30]:
# Flatten the cv_results column for easier viewing
results_df_flat = pd.concat([results_df.drop(columns=['cv_results']),
                             results_df['cv_results'].apply(pd.json_normalize)], axis=1)

print(results_df_flat)

                  model                                    params  mean_score  \
0  KNeighborsClassifier  {'n_neighbors': 3, 'weights': 'uniform'}    0.994178   
1    LogisticRegression               {'C': 0.1, 'penalty': 'l2'}    0.994178   

                                          cv_results  
0                                         mean_fi...  
1                                         mean_fi...  
